<h3> Creating Your First Graph and Running It in a Session </h3>

Um programa em TensorFlow é tipicamente separado em duas partes:
1. Construir um gráfo de computação
2. Executar o gráfo

In [2]:
#Importa biblioteca do TensorFlow
import tensorflow as tf

In [5]:
#Criandos o grafo de computação
x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")
f = x*x*y+y+2

In [14]:
#A sessão é responsável por carregar o grafo em dispositivos como cpu e gpu e executá-lo, e armazena os valores das variáveis
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [15]:
#Outra maneira de utilizar sessão
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    #Dessa maneira a sessão se fecha automaticamente ao final do bloco
print(result)

42


In [17]:
#Em vez de inicializar as variáveis uma por uma é possível iniciar todas de uma vez
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


In [19]:
#InteractiveSession é uma sessão que define ela mesmo como sessão default, então não é preciso de bloco
#Ainda é necessário fechar
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


<h3> Managing Graphs </h3>

In [23]:
#Qualquer nó que você cria é automaticamente adicionado ao gráfo default
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [27]:
#Você pode manipular multiplos gráfos independente, criando um novo gráfo e temporariamente fazendo default dentro de um bloco
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
x2.graph is graph

True

In [29]:
x2.graph is tf.get_default_graph()

False

DICA: Em Jupyter ou shell Python, é comum executar o mesmo comando várias vezes, como resultado você pode ter uma gráfo default contendo multiplos nós duplicados. Uma solução é restartar o kernel do Jupyter, mas a solução mas conveniente é ápenas resetar o gráfo default executando: tf.reset_default_graph()

<h3> Lifecycle of a Node Value </h3>

In [32]:
#Quando você avalia um nó, TensorFlow automaticamente determina os nós dependentes e automaticamente os avalia primeiramente
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval()) # 10
    print(z.eval()) # 15

10
15


NOTA: Dessa maneira os valores não são reaproveitados, ou seja, quando o código acima é executado w e x são avaliados duas vezes

Todos os nós do gráfos são deletados entre execuções de gráfos, com execção dos valores de variáveis, que são mantidos através da sessão. O ciclo de vida de uma variável começa quando executa o initializer e termina quando sessão é fechada.

In [35]:
#Para executar o código anterior eficientemente, você pode pedir para tensorflow avaliar y e z em apenas uma execução de gráfo
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val) # 10
    print(z_val) # 15

10
15


Atenção: Em processos singulares de TensorFlow, multiplas sessões não compartilham nenhum estado , mesmo que elas usem o mesmo  gráfo (cada sessão terá sua própria cópia de cada variável). Em TensorFlow distribuidos, o estado da variável é armazenado em servidores, não em sessões, então multiplas sessões podem compartilhar a mesma variável.

<h3> Linear Regression with TensorFlow </h3>

Constantes e variáveis não tem uma entrada *são chamadas source operations). A entrada e saída são vetores multidimensionais, chamados _tensores_. Assim como vetores do NumPy, tensores possuem tipo e formato.

In [8]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [9]:
#Pega os dados California housing dataset
housing = fetch_california_housing()
m, n = housing.data.shape
#Adiciona termo de bias x0 = 1
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
#Constroi o gráfo para método dos minímos quadrados (X^T.X)^-1.X^T.y
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()

O principal benefício da usar TensorFlow em vez do NumPy é que ele tem capacidade de rodar automaticamente em GPU (se instalado com suporte para GPU)

<h3> Implementing Gradient Descent</h3>

In [10]:
#Normalizar o dataset para melhor desempenho do gradiente
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [27]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


<h4> Manually Computing the Gradients</h4>

The following code should be fairly self-explanatory, except for a few new elements:

- The random_uniform() function creates a node in the graph that will generate a tensor containing random values, given its shape and value range, much like NumPy’s rand() function.
- The assign() function creates a node that will assign a new value to a variable. In this case, it implements the BatchGradient Descent step θ(next step) = θ – η∇θMSE(θ).
- The main loop executes the training step over and over again (n_epochs times), and every 100 iterations it prints out the current Mean Squared Error (mse). You should see the MSE go down at every iteration.

In [25]:
#Calculando o gradiente descendente manualmente
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.53737885
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293705


<h4> Using autodiff </h4>

O código anterior funciona bem , mas ele requer o cálculo das derivadas matematicamente. No caso da regressão linear é fácil de fazer, mas quando se trabalha com deep learning começa a complicar. Você pode usar _symbolic_ _differentiation_ para automaticamente encontrar as equações para as derivadas parciais, mas o resultado pode não ser a maneira mais eficiente.

Para entender, se for pego a função $f(x)=exp(exp(exp(x)))$ que possue a derivada $f'(x) = exp(x)*exp(exp(x))*exp(exp(exp(x)))$. Se o código for calcular exatamente como aparece não pode ser muito eficiente. A maneira mais eficiente seria calcular $exp(x)$, então $exp(exp(x))$ e então $exp(exp(exp(x)))$ e depois multiplicar. 

Pela maneira naive você teria que calcular exp(x) 9 vezes (f(x) e f'(x)). Com essa abordagem seriam necessários apenas 3.

E fica pior quando sua função é definida por um código arbritário. Você pode calcular as derivadas parciais do código a seguir? (nem tente!)

In [30]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

Felizmente, TensorFlow autodiff vem para ajudar! Ele pode automaticamente e eficientemente computar gradientes. Simplesmente troque os gradients = ... no código do gradiente descendente pela seguinte linha:

    gradients = tf.gradients(mse, [theta])[0]
   
   e  o código funcionará corretamente!

In [33]:
#Calculando o gradiente descendente com autodiff
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse,[theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


A função _gradients()_ pega a operação (no caso _mse_) e uma lista de variáveis (neste caso _theta_), e cria uma lista nde operações (uma por variável) para computar os gradientes da operação com relação a cada variável. Assim, o nó de gradientes calculará o vetor de gradiente do MSE com relação a theta.

Existem quatro abordagens para computar gradientes automaticamente. Eles estão sumarizados na Tabela 9-2. O TensorFlow utiliza _reverse-mode autodiff_, que é perfeito (eficientemente e acertivo) quando existem muitas entradas e poucas saídas, que frequentemente é o caso das redes neurais. Ele computa toda as derivadas parciais das saídas com relkação a todas as entradas em apenas $n_{\text{outputs}} + 1$ gráfos transversais

<img src="imagens/maincompgrap.png"> </img>

<h4> Using Optimizer </h4>

Além de computar os gradientes para você, o TensorFlow provêm um número de ótimos otimizadores, incluindo Gradient Descent Optimizer. Você pode simplesmente trocar a linha do código anterior com gradients = ... e training_op = .. com o seguinte código:

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [35]:
#Calculando o gradiente descendente com Gradient Desceendent Optimizer
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


Se você quiser utilizar um otimizador diferente, você pode apenas mudar uma linha. Por exemplo, você pode usar momentum optimizer (que freq converge mais rápido que o gradiente descendente), definindo um otimizador assim:

optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
momentum=0.9)

In [37]:
#Calculando o gradiente descendente com Momentum Optimizer
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.52731586
Epoch 200 MSE = 0.52441466
Epoch 300 MSE = 0.52432835
Epoch 400 MSE = 0.5243219
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.52432066
Epoch 700 MSE = 0.524321
Epoch 800 MSE = 0.52432126
Epoch 900 MSE = 0.524321


<h3> Feeding Data to the Training Algorithm </h3>

A maneira de trocar os valores de x e y em tempo de execução, é usando a função placeholder(). Esses nós são especiais porque eles não realizam computação, eles apenas emitem os dados que você entrega em tempo de execução. Se você não especificar os valores em tempo de execução para um placeholder, é levantado uma exceção.

Você pode formalmente escolher o tamanho dos dados ou colocar _None_ para dimensões de qualquer tamanho.

In [3]:
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

In [5]:
print(B_val_1)

[[6. 7. 8.]]


In [7]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


NOTA: Na verdade, você pode alimentar (feed) a saída de qualquer operação, não apenas placeholders. Neste caso o TensorFlow não avalaria essas operações, ele usa os valores que foram colocados no feed.

Para implementar o mini-batch Gradient Descent, nós precisamos trocar pouca coisa no código. Primeiro trocar a definição de x e y para placeholder()

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

E definir o número de lotes e computar o número de lotes.

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

Na fase de execução, pegar os mini-batchs um por um, e prôver os valores de x e  y pelo feed_dict, quando avaliar um nó que depende de ambos.

In [19]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

#Calculando o gradiente descendente com Momentum Optimizer
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

In [21]:
best_theta

array([[ 2.0714476 ],
       [ 0.8462012 ],
       [ 0.11558535],
       [-0.26835832],
       [ 0.32982782],
       [ 0.00608358],
       [ 0.07052915],
       [-0.87988573],
       [-0.8634251 ]], dtype=float32)

NOTA: nós não precisamos passar x e y quando avaliar theta, pois ele não depende deles

<h3> Saving and Restoring Models </h3>

Para salvar modelos do TensorFlow apenas crie um nó Saver no fim da fase de construção (depois que todos os nós forem criados, e na fase de execução, chame o método sava() para salvar o modelo, passando a sessão e caminho do arquivo de checkpoint.

In [29]:
tf.reset_default_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 2.7544262
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.5727805
Epoch 300 MSE = 0.5585007
Epoch 400 MSE = 0.54907
Epoch 500 MSE = 0.54228795
Epoch 600 MSE = 0.5373789
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.5312425
Epoch 900 MSE = 0.5293704


In [31]:
#Para recuperar o save
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


Por default, o Saver salva e recupera toda as variáveis com o seu próprimo nome, mas se for necessário um maior controle, você pode especificar quais variáveis salvar e recuperar, e qual nome você quer usar. Por exemplo, o código abaixo irá recuperar apenas a variável _theta_ com nome de _weights_

In [33]:
saver = tf.train.Saver({"weights": theta})

<h3> Visualizing the Graph and Training Curves Using TensorBoard </h3>

Nós já fizemos um gráfo de computação que treina um modelo de regressão linear usando mini-batch gradient descent, e salvamos em intervalos regulares. Mas ainda para visualizar estavamos usando o print(). Existe uma maneira melhor de visualização chamada TensorBorard, se você adicionar algumas estatísticas de treinamento, ele irá mostrar visualizações interativas no seu browser (ex. learning curves). Você também provê a definição do grafo e ele irá mostrar uma interface para navegar. É muito últil para indetificar erros no gráfo e achar bottlenecks.

In [40]:
#O primeiro passo é salvar o gráfo em um diretorio de log que o TensorBoard irá lêr
#CUIDADO, é necessário criar diretorios de logm diferentes cada vez que executar seu programa
#, caso contrário, ele irá dar um merge nas informações e bagunçar tudo
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
# ---------------------------------------------------------------------------------------

tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [41]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [42]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [43]:
with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                                     # not shown

In [44]:
file_writer.close()

In [45]:
best_theta

array([[ 2.070016  ],
       [ 0.8204561 ],
       [ 0.1173173 ],
       [-0.22739051],
       [ 0.3113402 ],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

comando para iniciar o TensorBoard: tensorboard --logdir tf_logs/

TIP: If you want to take a peek at the graph directly within Jupyter, you can use the show_graph() function available in the notebook
for this chapter. It was originally written by A. Mordvintsev in his great deepdream tutorial notebook. Another option is to install
E. Jang’s TensorFlow debugger tool which includes a Jupyter extension for graph visualization (and more).